In [ ]:
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')
import geopandas
import matplotlib.pyplot as plt
from shapely.geometry import Point, MultiPoint
from geopandas import GeoDataFrame

In [ ]:
Charter_Schools_Analysis = pd.read_pickle("Charter_Schools_Analysis_Out.pkl")  

In [ ]:
Charter_Schools_Analysis.head()

In [ ]:
Charter_Schools_Analysis['Latitude'] = Charter_Schools_Analysis['Latitude'].astype(float)
Charter_Schools_Analysis['Longitude'] = Charter_Schools_Analysis['Longitude'].astype(float)

In [ ]:
Charter_Schools_Analysis['First_Lat'] = (Charter_Schools_Analysis.groupby('Latitude').cumcount() == 0).astype(int)
Charter_Schools_Analysis = Charter_Schools_Analysis.loc[Charter_Schools_Analysis['First_Lat'] > 0]
Charter_Schools_Analysis = Charter_Schools_Analysis.reset_index(drop=True)

In [ ]:
from geopy.distance import geodesic

In [ ]:
Charter_Schools_Analysis['location_home'] = list(zip(Charter_Schools_Analysis['Latitude'], Charter_Schools_Analysis['Longitude']))

In [ ]:
Charter_Analysis_Neighbor = Charter_Schools_Analysis[['School', 'location_home', 'Latitude', 'Longitude', 'predicted_survival', 'Safe', 'ANN_TOTAL_ENROLL']]

In [ ]:
for i in range(len(Charter_Analysis_Neighbor)):
    distance_array = []
    d_i = Charter_Analysis_Neighbor['location_home'][i]
    for j in range(len(Charter_Analysis_Neighbor)):
        d = (geodesic(d_i, Charter_Analysis_Neighbor['location_home'][j]).miles)
        distance_array.append(d)
        
    distance_df = pd.DataFrame(distance_array)
    distance_df = distance_df.rename(columns={0: "Distance"})
        
    Charter_Analysis_Neighbor2 =  Charter_Analysis_Neighbor.join(distance_df, how='outer')
    Charter_Analysis_Neighbor3 = Charter_Analysis_Neighbor2.nsmallest(6, 'Distance')
    Charter_Analysis_Neighbor4 = Charter_Analysis_Neighbor3[0:6] 
    Charter_Analysis_Neighbor4 = Charter_Analysis_Neighbor4.reset_index(drop=True)
        
    Charter_Analysis_Neighbor4['Indexi'] = Charter_Analysis_Neighbor4.index
    Charter_Analysis_Neighbor4['Indexj'] = i
    Charter_Analysis_Neighbor5 = Charter_Analysis_Neighbor4.pivot(index= "Indexj", columns='Indexi')
   
    if i == 0:
        Charter_Schools_Wide = Charter_Analysis_Neighbor5
    else:
        Charter_Schools_Wide = Charter_Schools_Wide.append(Charter_Analysis_Neighbor5, ignore_index=True)

In [ ]:
Charter_Schools_Wide[0:20]

In [ ]:
Charter_Schools_Wide.to_csv('CP_Analysis.csv')